# Part 2: Using your Algorithm in Amazon SageMaker

Once you have your container packaged, you can use it to train models and use the model for hosting or batch transforms. Let's do that with the algorithm we made above.

## Set up the environment

Here we specify a bucket to use and the role that will be used for working with SageMaker.

In [1]:
# S3 prefix
prefix = 'DEMO-scikit-byo-iris'

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

## Create the session

The session remembers our connection parameters to SageMaker. We'll use it to perform all of our SageMaker operations.

In [2]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

## Upload the data for training

When training large models with huge amounts of data, you'll typically use big data tools, like Amazon Athena, AWS Glue, or Amazon EMR, to create your data in S3. For the purposes of this example, we're using some the classic [Iris dataset](https://en.wikipedia.org/wiki/Iris_flower_data_set), which we have included. 

We can use use the tools provided by the SageMaker Python SDK to upload the data to a default bucket. 

In [3]:
WORK_DIRECTORY = 'data'

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

## Create an estimator and fit the model

In order to use SageMaker to fit our algorithm, we'll create an `Estimator` that defines how to use the container to train. This includes the configuration we need to invoke SageMaker training:

* The __container name__. This is constructed as in the shell commands above.
* The __role__. As defined above.
* The __instance count__ which is the number of machines to use for training.
* The __instance type__ which is the type of machine to use for training.
* The __output path__ determines where the model artifact will be written.
* The __session__ is the SageMaker session object that we defined above.

Then we use fit() on the estimator to train against the data that we uploaded above.

In [4]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/sagemaker-decision-trees:latest'.format(account, region)

tree = sage.estimator.Estimator(image,
                       role, 1, 'ml.c4.2xlarge',
                       output_path="s3://{}/output".format(sess.default_bucket()),
                       sagemaker_session=sess)

tree.fit(data_location)

2019-11-27 02:08:02 Starting - Starting the training job...
2019-11-27 02:08:03 Starting - Launching requested ML instances......
2019-11-27 02:09:03 Starting - Preparing the instances for training...
2019-11-27 02:09:49 Downloading - Downloading input data
2019-11-27 02:09:49 Training - Downloading the training image...
2019-11-27 02:10:28 Uploading - Uploading generated training model
2019-11-27 02:10:28 Completed - Training job completed
Starting the training.
Training complete.
Training seconds: 50
Billable seconds: 50


## Hosting your model
You can use a trained model to get real time predictions using HTTP endpoint. Follow these steps to walk you through the process.

### Deploy the model

Deploying the model to SageMaker hosting just requires a `deploy` call on the fitted model. This call takes an instance count, instance type, and optionally serializer and deserializer functions. These are used when the resulting predictor is created on the endpoint.

In [5]:
from sagemaker.predictor import csv_serializer
predictor = tree.deploy(1, 'ml.m4.xlarge', serializer=csv_serializer)

--------------------------------------------------------------------------------------!

### Choose some data and use it for a prediction

In order to do some predictions, we'll extract some of the data we used for training and do predictions against it. This is, of course, bad statistical practice, but a good way to see how the mechanism works.

In [6]:
shape=pd.read_csv("data/iris.csv", header=None)
shape.sample(3)

,0,1,2,3,4
38,setosa,4.4,3.0,1.3,0.2
74,versicolor,6.4,2.9,4.3,1.3
11,setosa,4.8,3.4,1.6,0.2


In [7]:
# drop the label column in the training set
shape.drop(shape.columns[[0]],axis=1,inplace=True)
shape.sample(3)

,1,2,3,4
71,6.1,2.8,4.0,1.3
38,4.4,3.0,1.3,0.2
9,4.9,3.1,1.5,0.1


In [8]:
import itertools

a = [50*i for i in range(3)]
b = [40+i for i in range(10)]
indices = [i+j for i,j in itertools.product(a,b)]

test_data=shape.iloc[indices[:-1]]

Prediction is as easy as calling predict with the predictor we got back from deploy and the data we want to do predictions with. The serializers take care of doing the data conversions for us.

In [9]:
print(predictor.predict(test_data.values).decode('utf-8'))

setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
virginica
virginica
virginica
virginica
virginica
virginica
virginica
virginica
virginica



### Optional cleanup
When you're done with the endpoint, you'll want to clean it up.

In [10]:
sess.delete_endpoint(predictor.endpoint)

# Part 3: Run Batch Transform Job
You can use a trained model to get inference on large data sets by using [Amazon SageMaker Batch Transform](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html). A batch transform job takes your input data S3 location and outputs the predictions to the specified S3 output folder. Similar to hosting, you can extract inferences for training data to test batch transform.

### Create a Transform Job
We'll create an `Transformer` that defines how to use the container to get inference results on a data set. This includes the configuration we need to invoke SageMaker batch transform:

* The __instance count__ which is the number of machines to use to extract inferences
* The __instance type__ which is the type of machine to use to extract inferences
* The __output path__ determines where the inference results will be written

In [11]:
transform_output_folder = "batch-transform-output"
output_path="s3://{}/{}".format(sess.default_bucket(), transform_output_folder)

transformer = tree.transformer(instance_count=1,
                               instance_type='ml.m4.xlarge',
                               output_path=output_path,
                               assemble_with='Line',
                               accept='text/csv')

Using already existing model: sagemaker-decision-trees-2019-11-27-02-08-02-016


We use tranform() on the transfomer to get inference results against the data that we uploaded. You can use these options when invoking the transformer. 

* The __data_location__ which is the location of input data
* The __content_type__ which is the content type set when making HTTP request to container to get prediction
* The __split_type__ which is the delimiter used for splitting input data 
* The __input_filter__ which indicates the first column (ID) of the input will be dropped before making HTTP request to container

In [ ]:
transformer.transform(data_location, content_type='text/csv', split_type='Line', input_filter='$[1:]')
transformer.wait()

........

For more information on the configuration options, see [CreateTransformJob API](https://docs.aws.amazon.com/sagemaker/latest/dg/API_CreateTransformJob.html)

### View Output
Lets read results of above transform job from s3 files and print output

# [Optional] Movember Foundation

Hey there,

This Movember I'm growing my moustache for 30 days and Moving **10000 steps per day** for men's health. I'm doing it to raise funds for the Movember Foundation, and I need your support:

**https://mobro.co/13864320?mc=5**

I'm supporting the Movember Foundation because they’re tackling some of the most significant health issues faced by men. Your donations will help them stop men dying too young.

There are two ways you can contribute to my Mo growing effort:

Donate online at **https://mobro.co/13864320?mc=5**

Or, write a cheque to 'Movember' referencing my registration ID (13864320) and mail it to: Movember, PO Box 4141, MOUNT MAUNGANUI, 3149

Learn about the important work Movember is funding and the impact your donation will have: https://nz.movember.com/programs/cause?mc=5

There's a lot riding on this moustache, so thank you. I appreciate your support!

**Mo Bro Purnesh Tripathi**

In [ ]:
s3_client = sess.boto_session.client('s3')
s3_client.download_file(sess.default_bucket(), "{}/iris.csv.out".format(transform_output_folder), '/tmp/iris.csv.out')
with open('/tmp/iris.csv.out') as f:
    results = f.readlines()   
print("Transform results: \n{}".format(''.join(results)))